<a href="https://colab.research.google.com/github/atherfawaz/BERT-Supervised/blob/master/RoBERTa_Supervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/drive/My Drive/Ebryx/blogs_train

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
[Errno 2] No such file or directory: '/content/drive/My Drive/Ebryx/blogs_train'
/content


In [ ]:
# Code for TPU packages install
!curl -q https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

In [ ]:
!pip install -q transformers

# Importing stock libraries
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# Importing lackages from our NLP-Hugging Package
from transformers import RobertaConfig, RobertaModel, RobertaTokenizerFast, RobertaForSequenceClassification

In [2]:
# TPU
import torch_xla
import torch_xla.core.xla_model as xm
from sklearn.utils import shuffle
device = xm.xla_device()

In [14]:
%cd /content/gdrive/My Drive/Ebryx/blogs_train
# Creating the dataset and dataloader for the neural network
df = pd.read_csv('/content/gdrive/My Drive/Ebryx/dataset.csv')
df = df[['Post', 'Sign', 'Gender']]

encode_dict = {}

def encode_cat(x):
    if x not in encode_dict.keys():
      encode_dict[x]=len(encode_dict)
    return encode_dict[x]

df['Sign'] = df['Sign'].apply(lambda x: encode_cat(x))
df['Post'] = df['Post'].str.replace('\r','').str.replace('\t','').str.replace('\xa0', '').str.replace('urlLink', '')
df['Count'] = df['Post'].apply(lambda x: len(str(x).split()))

average_words = df['Count'].mean()
print('Average Words: ', average_words)

df = df [['Post', 'Sign']]

df = shuffle(df)
df.head(10)

/content/gdrive/My Drive/Ebryx/blogs_train
Average Words:  201.2948833788611


,Post,Sign
100817,waiting for mommy to get home so i can ...,10
23416,"It's named as ""Let's put the piece...",6
63355,Pete Guither explains that Bush plan...,0
123586,I need to speak to the Lindse...,4
300949,The Talk N= Natalie--9 year old I ba...,0
285056,2004-04-05 12:51:00; I think I'm gettin...,4
162268,God I hate Sunday's they are ...,8
122942,Wow kids... Everyone's posti...,4
256064,so i went for a walk with Carrie after ...,10
298381,Holy shit I really haven't blogge...,0


In [15]:
# Creating a CustomDataset class that is used to read the updated dataframe and tokenize the text. 
# The class is used in the return_dataloader function

class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        post = str(self.data.Post[index])
        post = " ".join(post.split())
        inputs = self.tokenizer.encode_plus(
            post,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.Sign[index], dtype=torch.float)
        } 
    
    def __len__(self):
        return self.len


# Creating the dataset and dataloader for the neural network

MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 8
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state=200)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = Triage(train_dataset, tokenizer, MAX_LEN)
testing_set = Triage(test_dataset, tokenizer, MAX_LEN)

train_params = {'batch_size': TRAIN_BATCH_SIZE,
            'shuffle': True,
            'num_workers': 1
            }

val_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 1
                }

training_loader = DataLoader(training_set, **train_params)
validation_loader = DataLoader(testing_set, **val_params)

FULL Dataset: (380720, 2)
TRAIN Dataset: (304576, 2)
TEST Dataset: (76144, 2)


In [16]:
# Creating the customized model, by adding a drop out and a dense layer on top of roberta to get the final output for the model. 

class ModelClass(torch.nn.Module):
    def __init__(self):
        super(ModelClass, self).__init__()
        self.model_layer = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 12)

    def forward(self, input_ids, attention_mask):
        output_1 = self.model_layer(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

# Function to return model based on the defination of Model Class
def return_model(device):
    model = ModelClass()
    model = model.to(device)
    return model


# Function to calcuate the accuracy of the model
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

# Epoch 1

In [17]:
#model = ModelClass()
#model.to(device)

#loss_function = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCHS):
  steps = 0
  n_tr_correct, n_val_correct = 0, 0
  tr_loss, val_loss = 0, 0
  nb_tr_examples, nb_val_examples = 0, 0
  for (i, data), (j, val_data) in zip(enumerate(training_loader, 0), enumerate(validation_loader, 0)):
      model.train()
      ids = data['ids'].to(device, dtype = torch.long)
      mask = data['mask'].to(device, dtype = torch.long)
      targets = data['targets'].to(device, dtype = torch.long)

      outputs = model(ids, mask).squeeze()
      optimizer.zero_grad()
      loss = loss_function(outputs, targets)
      big_val, big_idx = torch.max(outputs.data, dim=1)
      
      n_tr_correct += calcuate_accu(big_idx, targets)
      nb_tr_examples += targets.size(0)

      tr_loss += loss.item()
      
      if i%100==0:
        loss_step = tr_loss/100
        train_acc = n_tr_correct/nb_tr_examples
        tr_loss, n_tr_correct, nb_tr_examples = 0, 0, 0
        print(f"[{i}] train_loss: {loss_step:.3f} train_acc: {train_acc:.3f}", end="    ") 
      
      optimizer.zero_grad()
      loss.backward()
      
      # # When using GPU or GPU
      # optimizer.step()
      
      # When using TPU
      xm.optimizer_step(optimizer)
      xm.mark_step()

      model.eval()
      with torch.no_grad():
          ids = val_data['ids'].to(device, dtype = torch.long)
          mask = val_data['mask'].to(device, dtype = torch.long)
          targets = val_data['targets'].to(device, dtype = torch.long)

          outputs = model(ids, mask).squeeze()
          loss = loss_function(outputs, targets)
          big_val, big_idx = torch.max(outputs.data, dim=1)
          
          n_val_correct += calcuate_accu(big_idx, targets)
          nb_val_examples += targets.size(0)

          val_loss += loss.item()
          
          if j%100==0:
            val_step = val_loss/100
            val_acc = n_val_correct/nb_val_examples
            val_loss, n_val_correct, nb_val_examples = 0, 0, 0
            print(f"val_loss: {val_step:.3f} val_acc : {val_acc:.3f}") 

[0] train_loss: 0.025 train_acc: 0.125    val_loss: 0.025 val_acc : 0.000
[100] train_loss: 2.483 train_acc: 0.094    val_loss: 2.473 val_acc : 0.110
[200] train_loss: 2.481 train_acc: 0.095    val_loss: 2.475 val_acc : 0.111
[300] train_loss: 2.479 train_acc: 0.100    val_loss: 2.482 val_acc : 0.087
[400] train_loss: 2.477 train_acc: 0.094    val_loss: 2.483 val_acc : 0.101
[500] train_loss: 2.480 train_acc: 0.098    val_loss: 2.478 val_acc : 0.101
[600] train_loss: 2.481 train_acc: 0.094    val_loss: 2.464 val_acc : 0.128
[700] train_loss: 2.477 train_acc: 0.104    val_loss: 2.482 val_acc : 0.080
[800] train_loss: 2.476 train_acc: 0.107    val_loss: 2.470 val_acc : 0.105
[900] train_loss: 2.475 train_acc: 0.110    val_loss: 2.481 val_acc : 0.081
[1000] train_loss: 2.471 train_acc: 0.108    val_loss: 2.483 val_acc : 0.087
[1100] train_loss: 2.468 train_acc: 0.102    val_loss: 2.466 val_acc : 0.110
[1200] train_loss: 2.464 train_acc: 0.106    val_loss: 2.485 val_acc : 0.094
[1300] trai

# Epoch 2

In [ ]:
steps = 0
n_tr_correct, n_val_correct = 0, 0
tr_loss, val_loss = 0, 0
nb_tr_examples, nb_val_examples = 0, 0
for (i, data), (j, val_data) in zip(enumerate(training_loader, 0), enumerate(validation_loader, 0)):
    model.train()
    ids = data['ids'].to(device, dtype = torch.long)
    mask = data['mask'].to(device, dtype = torch.long)
    targets = data['targets'].to(device, dtype = torch.long)

    outputs = model(ids, mask).squeeze()
    optimizer.zero_grad()
    loss = loss_function(outputs, targets)
    big_val, big_idx = torch.max(outputs.data, dim=1)
    
    n_tr_correct += calcuate_accu(big_idx, targets)
    nb_tr_examples += targets.size(0)

    tr_loss += loss.item()
    
    if i%100==0:
      loss_step = tr_loss/100
      train_acc = n_tr_correct/nb_tr_examples
      tr_loss, n_tr_correct, nb_tr_examples = 0, 0, 0
      print(f"[{i}] train_loss: {loss_step:.3f} train_acc: {train_acc:.3f}", end="    ") 
    
    optimizer.zero_grad()
    loss.backward()
    
    # # When using GPU or GPU
    # optimizer.step()
    
    # When using TPU
    xm.optimizer_step(optimizer)
    xm.mark_step()

    model.eval()
    with torch.no_grad():
        ids = val_data['ids'].to(device, dtype = torch.long)
        mask = val_data['mask'].to(device, dtype = torch.long)
        targets = val_data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask).squeeze()
        loss = loss_function(outputs, targets)
        big_val, big_idx = torch.max(outputs.data, dim=1)
        
        n_val_correct += calcuate_accu(big_idx, targets)
        nb_val_examples += targets.size(0)

        val_loss += loss.item()
        
        if j%100==0:
          val_step = val_loss/100
          val_acc = n_val_correct/nb_val_examples
          val_loss, n_val_correct, nb_val_examples = 0, 0, 0
          print(f"val_loss: {val_step:.3f} val_acc : {val_acc:.3f}") 

[0] train_loss: 0.023 train_acc: 0.156    val_loss: 0.018 val_acc : 0.375
[100] train_loss: 2.213 train_acc: 0.237    val_loss: 2.258 val_acc : 0.239
[200] train_loss: 2.210 train_acc: 0.242    val_loss: 2.228 val_acc : 0.214
[300] train_loss: 2.222 train_acc: 0.243    val_loss: 2.261 val_acc : 0.210
[400] train_loss: 2.204 train_acc: 0.235    val_loss: 2.252 val_acc : 0.230
[500] train_loss: 2.208 train_acc: 0.239    val_loss: 2.208 val_acc : 0.239
[600] train_loss: 2.206 train_acc: 0.241    val_loss: 2.268 val_acc : 0.225
[700] train_loss: 2.167 train_acc: 0.248    val_loss: 2.262 val_acc : 0.211
[800] train_loss: 2.197 train_acc: 0.245    val_loss: 2.206 val_acc : 0.241
[900] train_loss: 2.198 train_acc: 0.264    val_loss: 2.173 val_acc : 0.249
[1000] train_loss: 2.186 train_acc: 0.247    val_loss: 2.156 val_acc : 0.264
[1100] train_loss: 2.205 train_acc: 0.238    val_loss: 2.176 val_acc : 0.250
[1200] train_loss: 2.174 train_acc: 0.250    val_loss: 2.254 val_acc : 0.215
[1300] trai